In [ ]:
%%bash
wget -nc -O data/robert_frost.txt https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [ ]:
import re
from copy import deepcopy

import numpy as np
from nltk import ngrams

In [ ]:
def add_plus_one(d, *args):
    if len(args) == 1:
        # end of recursion: update counter
        try:
            d[args[0]] += 1
        except KeyError:
            d[args[0]] = 1

        return d
    else:
        # dig deeper into the rabit hole
        if args[0] not in d:
            d[args[0]] = {}

        d[args[0]] = add_plus_one(d[args[0]], *args[1:])

    return d


d = {"A": {"B": {"C": 10}}, "D": {"B": {"C": 1}}}
print(d)
d = add_plus_one(d, "A", "B", "C")
print(d)
d = add_plus_one(d, "A", "B", "D")
print(d)
d = add_plus_one(d, "D", "B", "C")
print(d)

In [ ]:
initial = {}
first_order = {}
second_order = {}

n_lines_used = 0

with open("data/robert_frost.txt", "r") as ifn:
    for line in ifn:
        # Skip empty lines
        if len(line) == 0:
            continue

        # Extract tokens
        tokens = [tok.rstrip("\n").lower() for tok in re.findall(r"\w+", line)]
        if len(tokens) < 3:
            continue

        # Append terminating token to the line
        tokens.append("<END>")

        # Update initial
        initial = add_plus_one(initial, tokens[0])

        first_order = add_plus_one(first_order, *tokens[:2])

        for words in ngrams(tokens, 3):
            second_order = add_plus_one(second_order, *words)

        n_lines_used += 1

print(n_lines_used)
print(initial)
print(first_order)
print(second_order)

In [ ]:
# normalise
p_initial = deepcopy(initial)
intitial_total = sum(initial.values())
for k in initial:
    p_initial[k] /= intitial_total
p_initial

In [ ]:
# normalise
p_first_order = deepcopy(first_order)
for k in first_order:
    total = sum(first_order[k].values())
    for kk in first_order[k]:
        p_first_order[k][kk] /= total
p_first_order

In [ ]:
# normalise
p_second_order = deepcopy(second_order)
for k in second_order:
    for kk in second_order[k]:
        total = sum(second_order[k][kk].values())
        for kkk in second_order[k][kk]:
            p_second_order[k][kk][kkk] /= total
p_second_order

In [ ]:
def sample_word(d):
    return np.random.choice(list(d.keys()), p=list(d.values()))


def generate():
    sentence = []

    # initital word
    w0 = sample_word(p_initial)
    sentence.append(w0)

    # second word
    w1 = sample_word(p_first_order[w0])
    sentence.append(w1)

    # rest
    while True:
        w2 = sample_word(p_second_order[w0][w1])
        if w2 == "<END>":
            break

        sentence.append(w2)
        w0 = w1
        w1 = w2

    print(" ".join(sentence))


for _ in range(5):
    generate()